# Legal Word and Sentence Embeddings

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Legal/2.Embeddings.ipynb)

# Legal Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we use Spark NLP Legal Word (**roberta_embeddings_legal_roberta_base**) and Sentence (**sent_bert_base_uncased_legal**) Embeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

## Import Required Library

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Saving latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json to latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json


In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

👌 Detected license file /content/latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up John Snow Labs home in /home/ckl/.johnsnowlabs this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIBRARY_SECRET]spark-ocr/spark_ocr-4.0.1-py3-none-any.whl --force-reinstall"
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIBRARY_SECRET]spark-nlp-internal/spark_nlp_internal-4.1.0-py3-none-any.whl --force-reinstall"
Installing spark-nlp to /usr/bin/python3
Installing pyspark to /usr/bin/python3
Installed 3 products:
🕶 Spark-OCR installed! ✅ Empower your NLP with a set of eyes 
💊 Spark-Healthcare installed! ✅ Heal the planet with NLP! 
🚀 Spark-NLP installed! ✅ State of the art NLP at scale 
🍥 If you are on Google Colab, please restart your Notebook for cha

## Start Spark Session

In [ ]:
from johnsnowlabs import * 
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

👌 Detected license file /content/latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json
📋 Stored new John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_2_for_Spark-Healthcare_Spark-OCR.json
👌 Launched SparkSession with Jars for: 🚀Spark-NLP, 💊Spark-Healthcare, 🕶Spark-OCR


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# if you want to start the session with custom params as in start function above
def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")
      
    return builder.getOrCreate()

#spark = start(SECRET)


# Get sample text

In [ ]:
! pip install plotly

# Downloading sample datasets.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Legal/data/legal_pca_samples.csv



In [ ]:
df = pd.read_csv('legal_pca_samples.csv')

df.head()

,text,label
0,The fiscal year of the Company (herein called ...,fiscal-year
1,Each of the Borrower and each other member of ...,fiscal-year
2,Purchaser shall pay as the total Purchase Pric...,purchase-price
3,The purchase price to be paid by Purchaser to ...,purchase-price
4,The Guarantor hereby unconditionally and irrev...,guarantee


In [ ]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

+--------------------+--------------+
|                text|         label|
+--------------------+--------------+
|The fiscal year o...|   fiscal-year|
|Each of the Borro...|   fiscal-year|
|Purchaser shall p...|purchase-price|
|The purchase pric...|purchase-price|
|The Guarantor her...|     guarantee|
|The Holding Compa...|     guarantee|
|GFS will bear its...|      expenses|
|Each party shall ...|      expenses|
|Failure by either...|        waiver|
|Failure of any pa...|        waiver|
+--------------------+--------------+



# Sentence Embeddings

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_uncased_legal", "en") \
    .setInputCols("document") \
    .setOutputCol("document_embeddings")

sent_bert_base_uncased_legal download started this may take some time.
Approximate size to download 390.8 MB
[OK!]


# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [ ]:
# This class extracts the embeddings from the Spark NLP Annotation object
# from pyspark import ml as ML

class EmbeddingsUDF(
    Transformer, ML.param.shared.HasInputCol,  ML.param.shared.HasOutputCol,
    ML.util.DefaultParamsReadable, ML.util.DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: ML.linalg.Vectors.dense(vs), ML.linalg.VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("document_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = EmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
# import pyspark
pca =ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We did all process in one pipeline

pipeline = Pipeline().setStages([document_assembler, embeddings, embeddings_for_pca, pca])

In [ ]:
pipeline.getStages()

[DocumentAssembler_8abec4220f32,
 BERT_SENTENCE_EMBEDDINGS_dae49a767331,
 EmbeddingsUDF_0ac2049f116d,
 PCA_952f3b2a31a8]

In [ ]:
model = pipeline.fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result.select('pca_features', 'label').show(truncate=False)

+------------------------------------------------------------+--------------+
|pca_features                                                |label         |
+------------------------------------------------------------+--------------+
|[-11.77244716366699,-3.189947356388764,4.491162322931578]   |fiscal-year   |
|[-11.401110660624244,-3.769741415819742,3.240553405264421]  |fiscal-year   |
|[-4.7833146129210355,-0.49425080655375503,2.86978337765696] |purchase-price|
|[-5.455980892626393,-1.3412495828462314,3.3317290563506976] |purchase-price|
|[-8.84166062140954,-1.8203561928036347,0.1339206777916528]  |guarantee     |
|[-11.532893021254763,-2.4995736938288293,0.8141664251810977]|guarantee     |
|[-5.731707629428153,-3.8158639512483816,3.513596288588748]  |expenses      |
|[-3.8010514486947096,-4.345370568255292,1.6774341021337447] |expenses      |
|[-6.783526937000947,-5.815459791835023,3.2361589693958526]  |waiver        |
|[-7.139201238503948,-6.340440847367812,1.2347492798270943]  |wa

In [ ]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

,pca_features,label
0,"[-11.77244716366699, -3.189947356388764, 4.491...",fiscal-year
1,"[-11.401110660624244, -3.769741415819742, 3.24...",fiscal-year
2,"[-4.7833146129210355, -0.49425080655375503, 2....",purchase-price
3,"[-5.455980892626393, -1.3412495828462314, 3.33...",purchase-price
4,"[-8.84166062140954, -1.8203561928036347, 0.133...",guarantee
5,"[-11.532893021254763, -2.4995736938288293, 0.8...",guarantee
6,"[-5.731707629428153, -3.8158639512483816, 3.51...",expenses
7,"[-3.8010514486947096, -4.345370568255292, 1.67...",expenses
8,"[-6.783526937000947, -5.815459791835023, 3.236...",waiver
9,"[-7.139201238503948, -6.340440847367812, 1.234...",waiver


In [ ]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

,x,y,z,label
0,-11.772447,-3.189947,4.491162,fiscal-year
1,-11.401111,-3.769741,3.240553,fiscal-year
2,-4.783315,-0.494251,2.869783,purchase-price
3,-5.455981,-1.341250,3.331729,purchase-price
4,-8.841661,-1.820356,0.133921,guarantee
5,-11.532893,-2.499574,0.814166,guarantee
6,-5.731708,-3.815864,3.513596,expenses
7,-3.801051,-4.345371,1.677434,expenses
8,-6.783527,-5.815460,3.236159,waiver
9,-7.139201,-6.340441,1.234749,waiver


In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)

fig.show()

# Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols("document")\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"])\
    .setOutputCol("document_embeddings")

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[OK!]


In [ ]:
# Firstly we splitted the pipeline in two to get all token embeddings

pipeline = Pipeline().setStages([document_assembler, tokenizer, embeddings])

In [ ]:
model = pipeline.fit(sdf)

In [ ]:
result = model.transform(sdf)

In [ ]:
result_df = result.select("label", F.explode(F.arrays_zip("token.result", "document_embeddings.embeddings")).alias("cols"))\
                   .select(F.expr("cols['0']").alias("token"),
                           F.expr("cols['1']").alias("embeddings"),
                           "label")

result_df.show(truncate = 80)


+--------+--------------------------------------------------------------------------------+-----------+
|   token|                                                                      embeddings|      label|
+--------+--------------------------------------------------------------------------------+-----------+
|     The|[-0.1905832, 0.02907233, 0.1323556, 0.19562279, 0.7778327, 0.2899082, -0.1439...|fiscal-year|
|  fiscal|[-0.19621783, 0.14509645, 0.23111376, -0.5060165, -0.38397065, -0.16950995, 0...|fiscal-year|
|    year|[0.080063194, 0.22008342, 0.2320247, -0.4419714, 0.58936, -0.23692241, 0.1419...|fiscal-year|
|      of|[-0.14090316, 0.15613584, 0.24000195, -0.24493258, 0.8977557, 0.4878105, 0.09...|fiscal-year|
|     the|[-0.060954522, -0.0823207, 0.3149926, 0.12840015, -0.014585942, 0.97888047, -...|fiscal-year|
| Company|[-0.06074677, 0.27488348, 0.07146067, -0.39569902, 0.7331536, 0.8051565, 0.05...|fiscal-year|
|       (|[0.11098474, -0.23188351, 0.11235487, 0.07458271, 0.86

In [ ]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [ ]:
embeddings_for_pca = WordEmbeddingsUDF()

In [ ]:
DIMENSIONS  = 3

In [ ]:
# import pyspark
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

## Full Spark NLP + Spark MLLib pipeline

In [ ]:
# We run the second part of the pipeline

pipeline = Pipeline().setStages([embeddings_for_pca, pca])


In [ ]:
model = pipeline.fit(result_df)

In [ ]:
result = model.transform(result_df)

In [ ]:
result.select("token", "embeddings", "pca_features", "label").show(truncate = 60)

+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
|   token|                                                  embeddings|                                                pca_features|      label|
+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
|     The|[-0.1905832, 0.02907233, 0.1323556, 0.19562279, 0.7778327...|   [6.574772772813158,7.652864013547959,-0.9220383052178981]|fiscal-year|
|  fiscal|[-0.19621783, 0.14509645, 0.23111376, -0.5060165, -0.3839...|[5.2273970212221865,-0.0694013000887546,-1.3976500529690785]|fiscal-year|
|    year|[0.080063194, 0.22008342, 0.2320247, -0.4419714, 0.58936,...|   [9.185597653777974,1.0014780284057485,1.0998376175042803]|fiscal-year|
|      of|[-0.14090316, 0.15613584, 0.24000195, -0.24493258, 0.8977...|  [6.397928994152721,3.1917330471891896,-1.0405451015952547

In [ ]:
df = result.select('token', 'pca_features',  'label').toPandas()

df

,token,pca_features,label
0,The,"[6.574772772813158, 7.652864013547959, -0.9220...",fiscal-year
1,fiscal,"[5.2273970212221865, -0.0694013000887546, -1.3...",fiscal-year
2,year,"[9.185597653777974, 1.0014780284057485, 1.0998...",fiscal-year
3,of,"[6.397928994152721, 3.1917330471891896, -1.040...",fiscal-year
4,the,"[7.772300525397695, 8.005473064204317, -2.2717...",fiscal-year
...,...,...,...
672,constitute,"[6.1299452847326785, -1.5722393716218812, -0.5...",waiver
673,a,"[8.764674977819404, 1.0030854031878687, 0.4532...",waiver
674,continuing,"[4.15035950115174, -0.7722881120706406, -1.152...",waiver
675,waiver,"[7.82442790730159, -1.4749766528984671, -3.411...",waiver


In [ ]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "x", "y", "z", "label"]]

df

,token,x,y,z,label
0,The,6.574773,7.652864,-0.922038,fiscal-year
1,fiscal,5.227397,-0.069401,-1.397650,fiscal-year
2,year,9.185598,1.001478,1.099838,fiscal-year
3,of,6.397929,3.191733,-1.040545,fiscal-year
4,the,7.772301,8.005473,-2.271737,fiscal-year
...,...,...,...,...,...
672,constitute,6.129945,-1.572239,-0.582969,waiver
673,a,8.764675,1.003085,0.453227,waiver
674,continuing,4.150360,-0.772288,-1.152193,waiver
675,waiver,7.824428,-1.474977,-3.411841,waiver


In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()

That chart is super cool because you can see how the same token gets different embeddings depending on the context.